### Запуск QtLab

In [4]:
%run D:\qtlab_replacement\init.py
%matplotlib qt4
import matplotlib.pyplot as plt
from instruments import *
import numpy as np
import sweep
from save_pkl import *
import visa

In [5]:
from resonator_tools import circuit

In [6]:
import pylab
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LinearSegmentedColormap
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pickle

import numpy as np
#%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.ticker

# Спектроскопия

In [8]:
current_src = Yokogawa_GS210(address='gs210')
#current = Yokogawa_GS210(address='gs210')
pna = Agilent_N5242A('pna', address = 'PNA')
#lo1 = Agilent_E8257D('lo1', address = 'PSG1')

In [9]:
# векторный анализатор в режим линейного свипа
pna.set_sweep_mode("LIN")

True

## Резонаторная спектроскопия

In [28]:
current_src.set_current(100e-6)
pna.set_power(-40)

True

In [13]:
current_src.set_status(1)

True

In [14]:
pna.set_autoscale()

In [56]:
pna.set_xlim(3.8e9, 12e9)
pna.set_bandwidth(100)
pna.set_averages(1)
pna.set_nop(3001)
pna.set_power(-70)

True

In [57]:
res_data = pna.get_tracedata()
res_freqs = pna.get_freqpoints()
plt.figure("S21_amp")
plt.plot(res_freqs, 20*np.log10(res_data[0]))
plt.figure("S21_ph")
plt.plot(res_freqs, res_data[1])
#save_pkl_noplt('res_pna_', {"Amp": res_data[0] , "Ph": res_data[1], "freqs": res_freqs } )

c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


In [65]:
currents = np.linspace(-700e-6, 100e-6, 451)
pna.set_xlim(8e9, 10.5e9)
pna.set_bandwidth(30)
pna.set_averages(2)
pna.set_nop(1251)
pna.set_power(-70)
sweep.sweep(pna, (currents, current_src.set_current, 'Bias Coil current'), filename='M-M MLPX_2 8-10.5GHz_mode_anticross_-70dB')

Started at:  Jul 04 2017 01:48:19


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: Bias Coil current: 0.0001,	average cycle time:   44s	 44s			
Elapsed time: 5 h 31 m 21.56 s


{'S-parameter': (('Bias Coil current', 'Frequency'),
  (array([ -7.00000000e-04,  -6.98222222e-04,  -6.96444444e-04,
           -6.94666667e-04,  -6.92888889e-04,  -6.91111111e-04,
           -6.89333333e-04,  -6.87555556e-04,  -6.85777778e-04,
           -6.84000000e-04,  -6.82222222e-04,  -6.80444444e-04,
           -6.78666667e-04,  -6.76888889e-04,  -6.75111111e-04,
           -6.73333333e-04,  -6.71555556e-04,  -6.69777778e-04,
           -6.68000000e-04,  -6.66222222e-04,  -6.64444444e-04,
           -6.62666667e-04,  -6.60888889e-04,  -6.59111111e-04,
           -6.57333333e-04,  -6.55555556e-04,  -6.53777778e-04,
           -6.52000000e-04,  -6.50222222e-04,  -6.48444444e-04,
           -6.46666667e-04,  -6.44888889e-04,  -6.43111111e-04,
           -6.41333333e-04,  -6.39555556e-04,  -6.37777778e-04,
           -6.36000000e-04,  -6.34222222e-04,  -6.32444444e-04,
           -6.30666667e-04,  -6.28888889e-04,  -6.27111111e-04,
           -6.25333333e-04,  -6.23555556e-04,  -6.2

## Кубитная спектроскопия

In [5]:
#Кубит
q_id = 1

pna.set_average(0)
pna.set_averages(1)
pna.set_power(Qubits[q_id]['P'])
pna.set_nop(1)
pna.set_centerfreq(Qubits[q_id]['Fr'])
pna.set_bandwidth(5)

lo1.set_power(Qubits[q_id]['Pex'])
lo1.set_status(1)

NameError: name 'lo1' is not defined

### 1D скан при фиксированной мощности

In [11]:
#Измерение
ex_freqs = np.arange(6.5e9, 7.0e9, 1e6)
pna.set_power (-50)

header = '''#Parameter = Ex frequency
#Ex power = {:f}'''
try:
    ex1D = sweep.sweep(pna, (ex_freqs, lo1.set_frequency, 'Second tone freq'), filename = "ex_{:.1f}dBm".format(lo1.get_power()))
except KeyboardInterrupt: pass 

Started at:  Jun 28 2017 20:13:50


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: Second tone freq: 6.999e+09,	average cycle time: 0.47s			
Elapsed time: 0 h 3 m 57.02 s


### 2D скан (мощность второго тона, частота второго тона)

In [8]:
ex_freqs = np.arange(5.5e9, 7e9, 10e6)
ex_powers = np.arange(-20, 15, 1)

pna.set_power(-50)
pna.set_bandwidth(10)
#lo1.set_power(-20)

header = '''#Parameter =
#Ro power = {:f}'''.format( pna.get_power() )

lo1.set_status(1)
#ps.S21vs2DPar(pna, currents, current.set_current, ex_freqs, lo1.set_frequency, filename = "#2{:.1f}dBm".format(lo1.get_power()),\
#           header = header)
ex2D = sweep.sweep(pna, (ex_powers, lo1.set_power, 'Second tone power'), (ex_freqs, lo1.set_frequency, 'Second tone freq'), filename = "#2{:.1f}dBm".format(pna.get_power()),\
           header = header)
lo1.set_status(0)

Started at:  Apr 25 2017 17:16:14


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: Second tone power:     14,	Second tone freq: 6.99e+09,	average cycle time: 0.42s				
Elapsed time: 0 h 37 m 4.85 s


True

# Импульсы

### Создаём девайсы

In [6]:
#if not ('pna' in locals()): pna = Agilent_N5242A('pna', address = 'PNA')
#if not ('lo1' in locals()): lo1 = Agilent_E8257D('lo1', address = 'PSG1')
sa = Agilent_N9030A('pxa', address = 'PXA')
#sa = Signal_Hound_SA(name='SA124', serial=61660066)
#lo_ex = Labbrick(name='lo_ex', serial=15249)
lo_ex = pna
lo_ro = Labbrick(name='lo_ro', serial=15257)
awg_tek = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')
adc = Spectrum_M3i2132('adc')

root: WARNING  Tektronix_AWG5014 : get all not yet fully functional


### Загружаем импульсные скрипты всякие

In [7]:
import awg_digital
import awg_iq
import data_reduce
import imp
import fitting
import plotting
import save_pkl
import tomography as tomography_new
import tomography_legacy as tomography
import sweep
imp.reload(awg_digital)
imp.reload(awg_iq)
imp.reload(data_reduce)
imp.reload(fitting)
imp.reload(plotting)
imp.reload(save_pkl)
imp.reload(tomography)
imp.reload(tomography_new)
imp.reload(sweep)

<module 'sweep' from 'D:\\qtlab_replacement\\scripts\\sweep.py'>

### Осцилляторы

In [8]:
# Источник тока - в autorange
#current.set_autorange(1)
#Мощности гетеродинов, постоянные
#Мощность гетеродина для возбуждения 13-16 дБм
#lo_ex_pow = 14
lo_ex_pow = 10
#lo_ex = lo1
#lo_ex.set_status(1)
#lo_ex.set_power(lo_ex_pow)
#pna как lo2 (excitation)
#Port1 - выход
#Мощность гетеродина для считывания 10-13 дБм, с учётом разветвителя 13-16 dBm
lo_ro_pow = 10
lo_ro.set_power(lo_ro_pow)

lo_ex_pow = 16
lo_ex = pna
pna.write("OUTP ON")
pna.write("SOUR1:POW1:MODE ON")
pna.write("SOUR1:POW2:MODE OFF")
pna.set_sweep_mode("CW")
lo_ex.set_power(lo_ex_pow)

True

### AWGшки

In [9]:
# промежуточные частоты для гетеродинной схемы:
ex_if = 125e6
ro_if = 75e6
# клоки генератора и оцифровщика
ex_clock = 1e9
ro_clock = 1e9

rep_rate = 20e3 # частота повторений эксперимента

awg_tek.stop()
awg_tek.set_clock(ex_clock) # клок всех авгшк
awg_tek.set_nop(ex_clock/rep_rate) # репрейт нужно задавать по=хорошему только на управляющей,
awg_tek.run()
# а вот длину сэмплов, которая очевидно то же самое, нужно задавать на всех авгшках.
# хорошо, что сейчас она только одна.

# миксеры подключены к AWG5014 через 6 дБ аттенюаторы. Ставим амплитуду 2В.
for channel in range(1,5):
    awg_tek.set_amplitude(1., channel=channel)
# Выходы 1, 2 тектроникс: возбуждение (кубитный тон) через 6 дБ аттенюатор
# Выходы 3, 4 тектроникс: считывание (резонаторный тон) через 6 дБ аттенюатор
iq_ex = awg_iq.awg_iq(awg_tek, awg_tek, 1, 2, lo_ex)
iq_ro = awg_iq.awg_iq(awg_tek, awg_tek, 3, 4, lo_ro)
iq_ex.set_if(ex_if)
iq_ro.set_if(ro_if)
iq_ex.set_sideband_id(-1)
iq_ro.set_sideband_id(-1)
iq_ex.set_frequency(Qubits[1]['F01'])
iq_ro.set_frequency(Qubits[1]['Fr'])

# К первому маркере первого канала тектроникса подсоединён триггер оцифровщика.
# Надо подавать какой-нибудь триггер, что-ли.
# А ещё нужно подавать клок. К сожалению наш оцифровщик не может в 1ГГц клок, ему нужен 500МГц клок.
# Будем подавать 500 МГц с цифрового выхода тектроникса.
awg_tek.set_marker1_low(-1, channel=1)
awg_tek.set_marker2_low(-1, channel=1)
awg_tek.set_marker1_high(2, channel=1)
awg_tek.set_marker2_high(2, channel=1)
awg_tek.set_marker1_low(-0.1, channel=2)
awg_tek.set_marker2_low(-0.1, channel=2) 
awg_tek.set_marker1_high(0.1, channel=2) # с клока хватит и такой амплитуды
awg_tek.set_marker2_high(0.1, channel=2) # с клока хватит и такой амплитуды
ro_trg = awg_digital.awg_digital(awg_tek, 1)
osc_trg = awg_digital.awg_digital(awg_tek, 5)
ro_adc_clock = awg_digital.awg_digital(awg_tek, 2) #
osc_adc_clock = awg_digital.awg_digital(awg_tek, 6) #

# тупо клок
ro_adc_clock.set_waveform(np.asarray([0,1]*int(ro_adc_clock.get_nop()/2), dtype=int))
osc_adc_clock.set_waveform(np.asarray([0,1]*int(ro_adc_clock.get_nop()/2), dtype=int))
awg_channels = {'iq_ex':iq_ex, 'iq_ro':iq_ro, 'ro_trg':ro_trg, 'osc_trg':osc_trg}
trg_length = 4e-9
pg = tomography_new.pulses(awg_channels)

root: WARNING  Tektronix_AWG5014 : Clock set to 1000000000.0. This is not fully functional yet. To avoid problems, it is better not to change the clock during operation


### Оцифровщик

In [10]:
# настройки оцифровщика
adc.set_clock(ro_clock)
adc.reset()            
adc.set_timeout(10000)


#adc.set_spc_samplerate(smp_rate)
#External trigger
adc.set_trigger_ext0_level0(0) # logical 0 value to 200 mV
adc.set_trigger_ext0_level1(1300) # logical 1 value to 600 mV
adc.trigger_termination_50Ohm()
adc.trigger_mode_pos()
adc.set_trigger_ext0_pulsewidth(0)
adc.disable_trigger_output()
adc.select_channel01()
adc.set_multi_mode()
#adc.set_segmentsize()
adc.set_trigger_ORmask_tmask_ext0()
adc.set_trigger_ANDmask_tmask_ext0()
adc.set_trigger_ORmask_tmask_NO_ch0()
adc.set_trigger_ORmask_tmask_NO_ch1()
adc.set_trigger_ANDmask_tmask_NO_ch0()
adc.set_trigger_ANDmask_tmask_NO_ch1()
# усилитель на оцифровщике
adc.set_input_amp_ch0(50)
adc.set_input_amp_ch1(50)
adc.set_input_offset_ch0(0)
adc.set_input_offset_ch1(0)

#External clock
adc.set_reference_clock(adc.get_clock())

# "Измеритель средних прошедших импульсов"
# Просто усредняет всё по номеру сэмпла.
adc_reducer = data_reduce.data_reduce(adc)
adc_reducer.filters['Mean Voltage (AC)'] = data_reduce.mean_reducer_noavg(adc, 'Voltage', 0)
adc_reducer.filters['S21+'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, iq_ro.get_if())
adc_reducer.filters['S21-'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, -iq_ro.get_if())

### Калибровка миксеров

In [11]:
#sa.set_ref(0)
iq_ex.get_calibration(sa)
iq_ro.get_calibration(sa)

d:\qtlab\ReiData\data/calibrations//IQ-if1.2e+08-rf6.8e+09-sb--1.pkl
d:\qtlab\ReiData\data/calibrations//IQ-if7.5e+07-rf9.5e+09-sb--1.pkl


{'I': (0.14214119709207787+0.56210568750479994j),
 'Q': (0.47509376666832875+0.8487834389723673j),
 'dc': (0.030911081453283903+0.027587223783327491j),
 'num_sidebands': 7,
 'score': -5.8391499534830054}

### Длительность и амплитуда считывающих импульсов

In [12]:
def set_adc_nop(ro_adc_length):
    adc.stop()
    signal_size = int(np.ceil(4./3.*(ro_adc_length)*adc.get_clock()))
    nop = int( 2**np.ceil(np.log2(signal_size)) )
    posttrigger = nop*3/4
    
    print (nop)
    adc.set_nop(nop)
    adc.set_post_trigger(posttrigger)

In [13]:
ro_adc_length = 0.6e-6        # Вот такой длины считываем (опухоль Фёдорова!!)
ro_amplitude = 0.21
ro_dac_length = 0.11e-6
adc.set_nums(1000)
#adc.set_software_nums_multi(1)
set_adc_nop(ro_adc_length)

512


In [14]:
#tomography.set_sequence([tomography.ro_rect(ro_amplitude, ro_dac_length, awg_channels)], awg_channels)
plt.plot(np.mean(np.real(adc.measure()['Voltage']), axis=0))

In [14]:
plt.pcolormesh(np.real(adc.measure()['Voltage']));
plt.colorbar()

In [15]:
adc.stop()

### Прохождение считывающих испульсов разной амплитуды

In [16]:
def set_pulse_amplitude(x):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    pg.set_seq([pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, x)])
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (np.linspace(0, 1, 51), set_pulse_amplitude, 'Readout amplitude'), filename='Readout pulse passthrough')

Started at:  Jul 01 2017 12:17:21


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


First sweep...

D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:637: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


Time left: 0 h 0 m 0.0 s,	parameter values: Readout amplitude:      1,	average cycle time:  5.4s			
Elapsed time: 0 h 4 m 33.85 s


### Осцилляции Раби

In [14]:
ex_amplitude = 0.8
pause_length = 16e-9
lengths = np.linspace(0e-9, 2e-6, 201)
readout_begin = np.max(lengths)
def set_ex_length(length):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    sequence = [pg.p(None, readout_begin-length), 
                pg.p('iq_ex', length, pg.rect, ex_amplitude), 
                pg.p(None, pause_length), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (lengths, set_ex_length, 'Rabi pulse length'), filename='Rabi')
measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Rabi'}, measurement_fitted,annotation=annotation)

Started at:  Jun 30 2017 16:08:00


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


First sweep...

D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:637: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


KeyboardInterrupt: 

In [52]:
save_pkl.save_pkl({'type':'Rabi'}, measurement_fitted,annotation=annotation)

c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


### Осцилляции Рамзея

In [78]:
ex_amplitude = 0.8
pi2_pulse = 22e-9
pause_length = 16e-9
delays = np.linspace(0e-9, 2000e-9, 101)
target_freq_offset = 17e6

readout_begin = np.max(delays)+pi2_pulse*2

def set_delay(delay):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p(None, readout_begin-length),
                pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude), 
                pg.p(None, delay), 
                pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude*np.exp(1j*delay*target_freq_offset*2*np.pi)), 
                pg.p(None, delay), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
    
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (delays, set_delay, 'Ramsey delay'), filename='Ramsey')
#measurement = sweep.sweep(adc_sz, (delays, set_delay, 'Ramsey delay'), filename='Ramsey')
measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
#save_pkl({'type':'Ramsey'}, measurement_fitted, annotation=annotation)

Started at:  Jul 01 2017 15:04:57


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


First sweep...

D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:637: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


Time left: 0 h 0 m 0.0 s,	parameter values: Ramsey delay:  2e-06,	average cycle time:    7s	7s		
Elapsed time: 0 h 11 m 51.29 s
9.46878357392e-07 [  3.16715214e-08   8.08724760e-08   1.29024234e-06   1.49382192e-06] [  6.10986903e-09   8.79847498e-11   2.91425779e-08   4.58014318e-08]


In [79]:
save_pkl.save_pkl({'type':'Ramsey'}, measurement_fitted, annotation=annotation)

c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


### Затухание свободной индукции (T1)

In [63]:
ex_amplitude = 0.8
pi_pulse = 44e-9
pause_length = 16e-9
delays = np.linspace(0e-9, 10000e-9, 41)
readout_begin = np.max(delays)+pi_pulse
def set_delay(delay):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p(None, readout_begin-length),
                pg.p('iq_ex', pi_pulse, pg.rect, ex_amplitude), 
                pg.p(None, delay), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
    
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (delays, set_delay, 'delay'), filename='Decay')

measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_fit)
#save_pkl.save_pkl({'type':'Decay'}, measurement_fitted)
annotation = 'Decay: {0:4.6g} s'.format(fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Decay'}, measurement_fitted, annotation=annotation)

Started at:  Jun 29 2017 17:16:14


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: delay:  1e-05,	average cycle time:  4.1s	1s		
Elapsed time: 0 h 2 m 46.87 s


In [64]:
annotation = 'Decay: {0:4.6g} s'.format(fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Decay'}, measurement_fitted, annotation=annotation)

c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


### Rabi amplitude oscillations

In [18]:
ex_amps = np.linspace(0.,1.0,51)
pause_length = 32e-9
length = 32e-9
sigma = 8e-9
num_pulses = 4
readout_begin = np.max((length+pause_length)*num_pulses)
def set_ex_amp(amp):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p('iq_ex', pi_pulse, pg.gauss_hd, sigma, ex_amplitude, 0), 
                pg.p(None, readout_begin-length)]*num_pulses+\ 
               [pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
    
    pg.set_seq(sequence)
    
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (ex_amps, set_ex_amp, 'Rabi pulse amplitude'), filename='Rabi')

Started at:  Jun 29 2017 15:49:20


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: Rabi pulse amplitude:      1,	average cycle time:  3.4s			
Elapsed time: 0 h 2 m 53.01 s


### gaussian $\pi/2$ pulse amplitudes

In [118]:
#amp_x=0.554
#amp_y=0.554
#sigma = 2e-9
#amp_x=0.29
#amp_y=0.29
amp_x = 0.925
length = 32e-9
pause_length = 8e-9
sigma = 8e-9
amp_x_hd = 0.925
length_hd = 32e-9
sigma_hd = 8e-9
pause_length_hd = 8e-9
alpha_hd = -0.92e-9

### Ramsey with gauss pulses

In [14]:
delays = np.linspace(0e-9, 2000e-9, 201)
target_freq_offset = 20e6

def set_delay(delay):
    sequence = [pg.p(None, readout_begin-length),
                pg.p('iq_ex', pi2_pulse, pg.gauss_hd, sigma, ex_amplitude, 0), 
                pg.p(None, delay), 
                pg.p('iq_ex', pi2_pulse, pg.gauss_hd, sigma, ex_amplitude*np.exp(1j*delay*target_freq_offset*2*np.pi), 0), 
                pg.p(None, delay), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
    
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (delays, set_delay, 'Ramsey delay'), filename='Ramsey')
measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Ramsey'}, measurement_fitted, annotation=annotation)

Started at:  Jul 01 2017 18:27:49


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


First sweep...

KeyboardInterrupt: 

# Optimizing readout pulse settings

In [28]:
ex_calib_seq = [pg.p('iq_ex', length, pg.gauss_hd, amp_x, 0, sigma), pg.p(None, pause_length)]*2
ro_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
adc_sz = tomography_new.sz_measurer(adc, ex_calib_seq, ro_seq, pg)
adc.set_software_nums_multi(20)
adc_sz.repeat_samples = 1

target_min = lambda: 1-adc_sz.measure()['Calibrated fidelity binary']

class ro_pulse_generator:
    def __init__(self, adc_sz, ro_amplitude, ro_dac_length):
        self.ro_amplitude = ro_amplitude
        self.ro_dac_length = ro_dac_length
        self.adc_sz = adc_sz
    def make_ro_seq(self):
        adc_sz.ro_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', self.ro_dac_length, pg.rect, self.ro_amplitude)]
    def set_ro_length(self, ro_dac_length):
        self.ro_dac_length = ro_dac_length
        self.make_ro_seq()
    def set_ro_amplitude(self, ro_amplitude):
        self.ro_amplitude = ro_amplitude
        self.make_ro_seq()

ro_dac_lengths = np.linspace(0, 0.05e-6, 6)
ro_dac_amplitudes = np.linspace(0.8,1.0, 5)

ro_pulse_generator_inst = ro_pulse_generator(adc_sz, ro_amplitude, ro_dac_length)
solution, score = sweep.optimize(target_min, (ro_pulse_generator_inst.set_ro_length, 3e-8), 
                                             (ro_pulse_generator_inst.set_ro_amplitude, 0.9), maxfun=30)
ro_dac_length = solution[0]
ro_dac_amplitude = solution[1]
#
#sweep.sweep(adc_sz, (ro_dac_lengths, ro_pulse_generator_inst.set_ro_length, 'Readout pulse length'), 
#                    (ro_dac_amplitudes, ro_pulse_generator_inst.set_ro_amplitude, 'Readout pulse amplitude'))

[3e-08, 0.9]


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:637: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


[  3.00000000e-08   9.00000000e-01] 0.098758418258


D:\qtlab_replacement\scripts\tomography.py:123: RuntimeWarning: invalid value encountered in true_divide
  probabilities = hists/hist_all


[  3.15000000e-08   9.00000000e-01] 0.0979598255237
[  3.00000000e-08   9.45000000e-01] 0.0958115507041
[  3.15000000e-08   9.45000000e-01] 0.100552258593
[  3.03750000e-08   9.11250000e-01] 0.0975540163758
[  2.88750000e-08   9.56250000e-01] 0.0983902022521
[  3.08437500e-08   9.14062500e-01] 0.0964064125485
[  3.04687500e-08   9.47812500e-01] 0.0986554848473
[  3.03984375e-08   9.20390625e-01] 0.0979533329261
[  3.04218750e-08   9.29531250e-01] 0.0981042440391
[  3.01875000e-08   9.28125000e-01] 0.0970716351845
[  2.97656250e-08   9.43593750e-01] 0.097596010233
[  2.99296875e-08   9.40078125e-01] 0.0999534574901
[  3.00937500e-08   9.36562500e-01] 0.098459147376
[  3.02109375e-08   9.37265625e-01] 0.097134038348
[  3.01171875e-08   9.45703125e-01] 0.0968076206815
[  2.99062500e-08   9.53437500e-01] 0.0960191444277
[  2.97890625e-08   9.52734375e-01] 0.0932210900474
[  2.96250000e-08   9.56250000e-01] 0.0995379208245
[  2.98828125e-08   9.44296875e-01] 0.0937737287103
[  2.96718750e-0

### |0> & |1> readout calibration

In [119]:
ro_dac_length = 50e-9
ro_dac_amplitude = 0.13
ex_calib_seq = [pg.p('iq_ex', length, pg.gauss_hd, amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)]*2
#ro_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
ro_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, 1.0)]
adc_sz = tomography_new.sz_measurer(adc, ex_calib_seq, ro_seq, pg)
adc_sz.save_last_samples = True
adc_sz.repeat_samples = 1
adc.set_software_nums_multi(20)
print(adc_sz.measure())
#sz_filter = adc_sz.calibrate()
#print('ROC AUC:', adc_sz.calib_roc_auc, 'Fidelity:', adc_sz.calib_fidelity)
#print('ROC AUC binary:', adc_sz.calib_roc_auc_binary, 'Fidelity binary:', adc_sz.calib_fidelity_binary) 

D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:637: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


{'Calibrated ROC AUC binary': 0.79882500000000001, 'Calibrated ROC AUC': 0.8531451324999999, 'Calibrated fidelity': 0.69208511117086002, 'Calibrated fidelity binary': 0.89376058179234064}


D:\qtlab_replacement\scripts\tomography.py:123: RuntimeWarning: invalid value encountered in true_divide
  probabilities = hists/hist_all


In [109]:
plt.figure()
plt.pcolormesh(np.real(adc_sz.samples[0,:,:]), vmin=15, vmax=25); plt.colorbar()
plt.figure()
plt.pcolormesh(np.real(adc_sz.samples[1,:,:]), vmin=15, vmax=25); plt.colorbar()

In [83]:
plt.hist(adc_sz.calib_pred.T, bins=20)

([array([  1.00000000e+01,   3.20000000e+01,   1.52000000e+02,
           5.35000000e+02,   1.33800000e+03,   2.55200000e+03,
           3.62600000e+03,   3.64800000e+03,   3.15300000e+03,
           2.06300000e+03,   1.18300000e+03,   7.36000000e+02,
           4.68000000e+02,   2.94000000e+02,   1.42000000e+02,
           5.20000000e+01,   8.00000000e+00,   7.00000000e+00,
           0.00000000e+00,   1.00000000e+00]),
  array([  0.00000000e+00,   8.00000000e+00,   2.10000000e+01,
           6.80000000e+01,   1.92000000e+02,   3.42000000e+02,
           6.31000000e+02,   9.23000000e+02,   1.40400000e+03,
           2.16500000e+03,   3.07900000e+03,   3.69000000e+03,
           3.34200000e+03,   2.34600000e+03,   1.16700000e+03,
           4.60000000e+02,   1.23000000e+02,   3.30000000e+01,
           5.00000000e+00,   1.00000000e+00])],
 array([-2.75677142, -2.46279575, -2.16882008, -1.87484441, -1.58086873,
        -1.28689306, -0.99291739, -0.69894172, -0.40496605, -0.11099037,
   

In [28]:
#plt.scatter(np.real(adc_sz.calib_usl_pred[0,:]), np.imag(adc_sz.calib_usl_pred[0,:]), color='r', s=1)
#plt.scatter(np.real(adc_sz.calib_usl_pred[1,:]), np.imag(adc_sz.calib_usl_pred[1,:]), color='b', s=1)
#plt.figure()
#plt.hist(adc_sz.calib_pred.T, bins=20)
plt.figure()
#plt.hist(np.real(adc_sz.calib_usl_pred.T), bins=20)
plt.plot(np.real(adc_sz.calib_bg-np.mean(adc_sz.calib_bg)))
plt.plot(np.imag(adc_sz.calib_bg-np.mean(adc_sz.calib_bg)))
plt.plot(np.real(adc_sz.calib_feature))
plt.plot(np.imag(adc_sz.calib_feature))

### Tomography

In [120]:
observables = { 'X': 0.5*np.asarray([[0, 1],   [1, 0]]),
                'Y': 0.5*np.asarray([[0, -1j],   [1j, 0]]),
                '-X': 0.5*np.asarray([[0, -1],   [-1, 0]]),
                '-Y': 0.5*np.asarray([[0, 1j],   [-1j, 0]]),
                'Z': 0.5*np.asarray([[1, 0],   [0, -1]])}
proj_seq = {'Xo':{'pulses': [pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, alpha_hd), pg.p(None, pause_length)]+ro_seq, 
                 'operator': observables['X']},
            'Yo':{'pulses': [pg.p('iq_ex', length, pg.gauss_hd, amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)]+ro_seq,
                 'operator': observables['Y']},
            '-Xo':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, 0, amp_x, sigma, alpha_hd), pg.p(None, pause_length)]+ro_seq,
                 'operator': observables['-X']},
            '-Yo':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, -amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)]+ro_seq,
                 'operator': observables['-Y']},
            'Zo': {'pulses':ro_seq, 'operator':observables['Z']} }
reconstruction_basis={'x':{'operator':observables['X']},
                      'y':{'operator':observables['Y']},
                      'z':{'operator':observables['Z']}}
tomo = tomography_new.tomography(adc_sz, pg, proj_seq, reconstruction_basis=reconstruction_basis)

In [121]:
proj_seq = {'Z': {'pulses':ro_seq, 'operator':observables['Z']}}
tomoz = tomography_new.tomography(adc_sz, pg, proj_seq)

In [122]:
## tomography of |0> state
adc.set_software_nums_multi(20)
tomo.set_prepare_seq([])
tomo.measure()

D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:637: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


{'-Xo': array(0.00175),
 '-Yo': array(0.00205),
 'Xo': array(-0.001),
 'Yo': array(-0.00145),
 'Zo': array(-0.29305),
 'x': -0.0013749999999999999,
 'y': -0.0017500000000000003,
 'z': -0.29305000000000003}

In [123]:
## tomography of |1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, alpha_hd), pg.p(None, pause_length)]*2)
tomo.measure()

D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:637: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


{'-Xo': array(-0.00845),
 '-Yo': array(-0.00925),
 'Xo': array(-0.01045),
 'Yo': array(-0.01145),
 'Zo': array(0.278),
 'x': -0.00099999999999999894,
 'y': -0.0010999999999999979,
 'z': 0.27800000000000008}

In [114]:
## tomography of |1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, alpha_hd), pg.p(None, pause_length)]+
                     [pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, alpha_hd), pg.p(None, pause_length),
                     pg.p('iq_ex', length, pg.gauss_hd, 0, amp_x, sigma, alpha_hd), pg.p(None, pause_length)]*10)
tomo.measure()

D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:637: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


{'-Xo': array(-0.16495),
 '-Yo': array(0.0142),
 'Xo': array(0.16615),
 'Yo': array(-0.0108),
 'Zo': array(-0.0291),
 'x': 0.16554999999999997,
 'y': -0.012499999999999997,
 'z': -0.029100000000000008}

In [117]:
## tomography of |1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, -0.92e-9), pg.p(None, pause_length)]+
                     [pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, -0.92e-9), pg.p(None, pause_length),
                     pg.p('iq_ex', length, pg.gauss_hd, 0, amp_x, sigma, -0.92e-9), pg.p(None, pause_length)]*10)
tomo.measure()

D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:637: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


{'-Xo': array(-0.1645),
 '-Yo': array(-0.0024),
 'Xo': array(0.16585),
 'Yo': array(-0.0056),
 'Zo': array(-0.02615),
 'x': 0.16517499999999999,
 'y': -0.0015999999999999994,
 'z': -0.026150000000000007}

In [43]:
## tomography of |0>+|1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma), pg.p(None, pause_length)])
tomo.measure()

D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:637: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


{'-Xo': array(-0.3375),
 '-Yo': array(-0.09515),
 'Xo': array(0.2783),
 'Yo': array(0.02285),
 'Zo': array(-0.03185),
 'x': 0.30789999999999995,
 'y': 0.059000000000000004,
 'z': -0.03185000000000001}

In [52]:
## tomography of |0>-|1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, 1j*amp_x, sigma), pg.p(None, pause_length)])
tomo.measure()

D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:637: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


{'-X': array(-0.05568),
 '-Y': array(0.30536),
 'X': array(0.02668),
 'Y': array(-0.32728),
 'Z': array(-0.02364)}

In [53]:
## tomography of -|0>-|1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma), pg.p(None, pause_length)])
tomo.measure()

D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:637: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


{'-X': array(-0.32804),
 '-Y': array(-0.05428),
 'X': array(0.30984),
 'Y': array(0.02416),
 'Z': array(-0.018)}

In [54]:
## tomography of -|0>+|1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -1j*amp_x, sigma), pg.p(None, pause_length)])
tomo.measure()

D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:637: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


{'-X': array(0.0272),
 '-Y': array(-0.32916),
 'X': array(-0.05404),
 'Y': array(0.30652),
 'Z': array(-0.01412)}

# Randomized benchmarking

In [127]:
import interleaved_benchmarking
imp.reload(interleaved_benchmarking)

bench = interleaved_benchmarking.interleaved_benchmarking(tomo)

unitaries = {0:np.sqrt(0.5)*np.asarray([[1, -1j],  [-1j, 1]]),\
             1:np.sqrt(0.5)*np.asarray([[1, -1],    [1, 1]]),\
             2:np.sqrt(0.5)*np.asarray([[1, 1j],   [1j, 1]]),\
             3:np.sqrt(0.5)*np.asarray([[1, 1],   [-1, 1]])}
bench.interleavers = {'X': {'pulses':[pg.p('iq_ex', length, pg.gauss_hd, -amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)],
                            'unitary': np.sqrt(0.5)*np.asarray([[1, -1j],  [-1j, 1]])},
                      'Y': {'pulses':[pg.p('iq_ex', length, pg.gauss_hd, 1j*amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)],
                            'unitary': np.sqrt(0.5)*np.asarray([[1, -1],    [1, 1]])},
                      '-X':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)],
                            'unitary': np.sqrt(0.5)*np.asarray([[1, 1j],   [1j, 1]])},
                      '-Y':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, -1j*amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)],
                            'unitary': np.sqrt(0.5)*np.asarray([[1, 1],   [-1, 1]])},
                      'I':{'pulses':[], 'unitary':np.asarray([[1, 0], [0,1]])}}
bench.sequence_length = 10
bench.random_sequence_num = 6
bench.prepare_random_interleaving_sequences()
#bench.generate_interleaver_sequence_from_names()

In [128]:
seq_lengths = np.arange(101)
sweep.sweep(bench, (seq_lengths, bench.set_sequence_length_and_regenerate, 'Gate number'))

Started at:  Jul 03 2017 19:26:04


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


First sweep...Started at:  Jul 03 2017 19:26:23
First sweep...

D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:637: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


Time left: 0 h 0 m 0.0 s,	parameter values: Interleaving sequence id:      5,	average cycle time:   31s	s	
Elapsed time: 0 h 3 m 6.67 s


ValueError: could not broadcast input array from shape (6,0) into shape (6,20)

In [22]:
bench.target_gate = bench.interleavers['X']['pulses']
bench.target_gate_unitary = bench.interleavers['X']['unitary']
bench.target_gate_name = 'X (benchmarking)'
tfunc = lambda : np.mean(bench.measure()['Euclidean distance'])
class hd_pulse_generator:
    def __init__(self, pulse_setter, pg, channel):
        self.amp_x = -amp_x_hd
        self.amp_y = amp_x_hd
        self.sigma = sigma_hd
        self.length = length_hd
        self.pause_length = pause_length_hd
        self.alpha = alpha_hd
        
        self.pg = pg
        self.channel = channel
        self.pulse = self.pulse_gen()
        self.pulse_setter = pulse_setter
    def set_real_amplitude(self,x):
        self.amp_x = x
        self.pulse_setter(self.pulse_gen())
    def set_imag_amplitude(self,y):
        self.amp_y = y
        self.pulse_setter(self.pulse_gen())
    def set_sigma(self,s):
        self.sigma = s
        self.pulse_setter(self.pulse_gen())
    def set_length(self,l):
        self.length = l
        self.pulse_setter(self.pulse_gen())
    def set_pause_length(self, p):
        self.pause_length = p
        self.pulse_setter(self.pulse_gen())
    def set_alpha(self, a):
        self.alpha = a
        self.pulse_setter(self.pulse_gen())
    def pulse_gen(self):
        return [pg.p(self.channel, self.length, self.pg.gauss_hd, self.amp_x+1j*self.amp_y, 0, self.sigma, self.alpha), pg.p(None, self.pause_length)]
    
xg = hd_pulse_generator(bench.set_target_pulse, pg, 'iq_ex')

In [ ]:
sol, score = sweep.optimize(tfunc,  (xg.set_real_amplitude, xg.amp_x), 
                                    (xg.set_imag_amplitude, xg.amp_y),
                                    (xg.set_alpha, xg.alpha))

In [23]:
amps = np.linspace(0.925, 0.975, 21)
alphas = np.linspace(-2e-9, 2e-9, 11)
sweep.sweep(bench, (amps, xg.set_real_amplitude, 'Amplitude'), (alphas, xg.set_alpha, 'HD DRAG coeff'), filename='Randomized interleaved benchmarging X-rotation')

Started at:  Jul 02 2017 23:03:34


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


First sweep...Started at:  Jul 02 2017 23:03:35
First sweep...

D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:637: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


Time left: 0 h 0 m 0.0 s,	parameter values: Interleaving sequence id:      5,	average cycle time:   31s	s	
Elapsed time: 0 h 3 m 4.35 s
Time left: 12 h 56 m 8.95 s,	parameter values: Amplitude:  0.925,	HD DRAG coeff: -2e-09,	average cycle time: 2e+02s	Started at:  Jul 02 2017 23:06:57
Time left: 0 h 0 m 0.0 s,	parameter values: Interleaving sequence id:      5,	average cycle time:   31s	s	
Elapsed time: 0 h 3 m 6.03 s
Time left: 12 h 53 m 44.37 s,	parameter values: Amplitude:  0.925,	HD DRAG coeff: -1.6e-09,	average cycle time: 2e+02s	Started at:  Jul 02 2017 23:10:20
Time left: 0 h 0 m 0.0 s,	parameter values: Interleaving sequence id:      5,	average cycle time:   31s	s	
Elapsed time: 0 h 3 m 6.96 s
Time left: 12 h 52 m 28.56 s,	parameter values: Amplitude:  0.925,	HD DRAG coeff: -1.2e-09,	average cycle time: 2e+02s	Started at:  Jul 02 2017 23:13:45
Time left: 0 h 0 m 0.0 s,	parameter values: Interleaving sequence id:      5,	average cycle time:   29s	s	
Elapsed time: 0 h 2 m 56.35 s

Time left: 9 h 58 m 23.85 s,	parameter values: Amplitude: 0.9375,	HD DRAG coeff: -1.6e-09,	average cycle time: 2.1e+02s	Started at:  Jul 03 2017 02:19:37
Time left: 0 h 0 m 0.0 s,	parameter values: Interleaving sequence id:      5,	average cycle time:   28s	s	
Elapsed time: 0 h 2 m 50.97 s
Time left: 9 h 56 m 0.73 s,	parameter values: Amplitude: 0.9375,	HD DRAG coeff: -1.2e-09,	average cycle time: 2.1e+02s	Started at:  Jul 03 2017 02:23:24
Time left: 0 h 0 m 0.0 s,	parameter values: Interleaving sequence id:      5,	average cycle time:   28s	s	
Elapsed time: 0 h 2 m 50.53 s
Time left: 9 h 53 m 37.34 s,	parameter values: Amplitude: 0.9375,	HD DRAG coeff: -8e-10,	average cycle time: 2.1e+02s	Started at:  Jul 03 2017 02:27:13
Time left: 0 h 0 m 0.0 s,	parameter values: Interleaving sequence id:      5,	average cycle time:   28s	s	
Elapsed time: 0 h 2 m 50.78 s
Time left: 9 h 51 m 12.96 s,	parameter values: Amplitude: 0.9375,	HD DRAG coeff: -4e-10,	average cycle time: 2.1e+02s	Started at: 

Time left: 7 h 27 m 53.54 s,	parameter values: Amplitude:   0.95,	HD DRAG coeff: -1.2e-09,	average cycle time: 2.3e+02s	Started at:  Jul 03 2017 06:12:30
Time left: 0 h 0 m 0.0 s,	parameter values: Interleaving sequence id:      5,	average cycle time:   29s	s	
Elapsed time: 0 h 2 m 54.21 s
Time left: 7 h 24 m 51.4 s,	parameter values: Amplitude:   0.95,	HD DRAG coeff: -8e-10,	average cycle time: 2.3e+02s	Started at:  Jul 03 2017 06:17:02
Time left: 0 h 0 m 0.0 s,	parameter values: Interleaving sequence id:      5,	average cycle time:   29s	s	
Elapsed time: 0 h 2 m 52.91 s
Time left: 7 h 21 m 47.56 s,	parameter values: Amplitude:   0.95,	HD DRAG coeff: -4e-10,	average cycle time: 2.3e+02s	Started at:  Jul 03 2017 06:21:34
Time left: 0 h 0 m 0.0 s,	parameter values: Interleaving sequence id:      5,	average cycle time:   29s	s	
Elapsed time: 0 h 2 m 53.5 s
Time left: 7 h 18 m 43.52 s,	parameter values: Amplitude:   0.95,	HD DRAG coeff:      0,	average cycle time: 2.3e+02s	Started at:  Ju

Time left: 4 h 19 m 15.38 s,	parameter values: Amplitude: 0.9625,	HD DRAG coeff: -8e-10,	average cycle time: 2.5e+02s	Started at:  Jul 03 2017 10:50:16
Time left: 0 h 0 m 0.0 s,	parameter values: Interleaving sequence id:      5,	average cycle time:   29s	s	
Elapsed time: 0 h 2 m 51.98 s
Time left: 4 h 15 m 25.7 s,	parameter values: Amplitude: 0.9625,	HD DRAG coeff: -4e-10,	average cycle time: 2.5e+02s	Started at:  Jul 03 2017 10:55:26
Time left: 0 h 0 m 0.0 s,	parameter values: Interleaving sequence id:      5,	average cycle time:   29s	s	
Elapsed time: 0 h 2 m 52.46 s
Time left: 4 h 11 m 35.46 s,	parameter values: Amplitude: 0.9625,	HD DRAG coeff:      0,	average cycle time: 2.5e+02s	Started at:  Jul 03 2017 11:00:37
Time left: 0 h 0 m 0.0 s,	parameter values: Interleaving sequence id:      5,	average cycle time:   29s	s	
Elapsed time: 0 h 2 m 52.91 s
Time left: 4 h 7 m 44.75 s,	parameter values: Amplitude: 0.9625,	HD DRAG coeff:  4e-10,	average cycle time: 2.5e+02s	Started at:  Jul 

Time left: 0 h 27 m 10.61 s,	parameter values: Amplitude:  0.975,	HD DRAG coeff: -4e-10,	average cycle time: 2.7e+02s	Started at:  Jul 03 2017 16:02:43
Time left: 0 h 0 m 0.0 s,	parameter values: Interleaving sequence id:      5,	average cycle time:   29s	s	
Elapsed time: 0 h 2 m 56.16 s
Time left: 0 h 22 m 40.75 s,	parameter values: Amplitude:  0.975,	HD DRAG coeff:      0,	average cycle time: 2.7e+02s	Started at:  Jul 03 2017 16:08:41
Time left: 0 h 0 m 0.0 s,	parameter values: Interleaving sequence id:      5,	average cycle time:   29s	s	
Elapsed time: 0 h 2 m 56.53 s
Time left: 0 h 18 m 10.13 s,	parameter values: Amplitude:  0.975,	HD DRAG coeff:  4e-10,	average cycle time: 2.7e+02s	Started at:  Jul 03 2017 16:14:40
Time left: 0 h 0 m 0.0 s,	parameter values: Interleaving sequence id:      5,	average cycle time:   29s	s	
Elapsed time: 0 h 2 m 56.2 s
Time left: 0 h 13 m 38.76 s,	parameter values: Amplitude:  0.975,	HD DRAG coeff:  8e-10,	average cycle time: 2.7e+02s	Started at:  Jul

{'-Xo': (('Amplitude', 'HD DRAG coeff', 'Interleaving sequence id'),
  (array([ 0.925 ,  0.9275,  0.93  ,  0.9325,  0.935 ,  0.9375,  0.94  ,
           0.9425,  0.945 ,  0.9475,  0.95  ,  0.9525,  0.955 ,  0.9575,
           0.96  ,  0.9625,  0.965 ,  0.9675,  0.97  ,  0.9725,  0.975 ]),
   array([ -2.00000000e-09,  -1.60000000e-09,  -1.20000000e-09,
           -8.00000000e-10,  -4.00000000e-10,   0.00000000e+00,
            4.00000000e-10,   8.00000000e-10,   1.20000000e-09,
            1.60000000e-09,   2.00000000e-09]),
   array([0, 1, 2, 3, 4, 5])),
  array([[[-0.17195,  0.13085,  0.06385,  0.06715, -0.1106 ,  0.1475 ],
          [-0.1839 ,  0.14225,  0.09035,  0.0517 , -0.11155,  0.13905],
          [-0.204  ,  0.1579 ,  0.1222 ,  0.03645, -0.1186 ,  0.1256 ],
          ..., 
          [-0.21325,  0.22255,  0.22795, -0.0618 , -0.1397 ,  0.101  ],
          [-0.20535,  0.2328 ,  0.23615, -0.08635, -0.1438 ,  0.11495],
          [-0.1952 ,  0.22595,  0.23275, -0.1063 , -0.1555 ,  0

In [40]:
data = save_pkl.load_pkl('Euclidean distance Randomized interleaved benchmarging X-rotation', 'D:\\qtlab\\ReiData\\data\\2017-07-02\\23-03-34')
plt.figure()
plt.pcolormesh(data[1]['Euclidean distance'][1][1], data[1]['Euclidean distance'][1][0], data[1]['Euclidean distance'][2][:,:,5])
plt.colorbar()

D:\qtlab\ReiData\data\2017-07-02\23-03-34/Euclidean distance Randomized interleaved benchmarging X-rotation.pkl


In [33]:
data[1]

{'-Xo': (('Amplitude', 'HD DRAG coeff', 'Interleaving sequence id'),
  (array([ 0.925 ,  0.9275,  0.93  ,  0.9325,  0.935 ,  0.9375,  0.94  ,
           0.9425,  0.945 ,  0.9475,  0.95  ,  0.9525,  0.955 ,  0.9575,
           0.96  ,  0.9625,  0.965 ,  0.9675,  0.97  ,  0.9725,  0.975 ]),
   array([ -2.00000000e-09,  -1.60000000e-09,  -1.20000000e-09,
           -8.00000000e-10,  -4.00000000e-10,   0.00000000e+00,
            4.00000000e-10,   8.00000000e-10,   1.20000000e-09,
            1.60000000e-09,   2.00000000e-09]),
   array([0, 1, 2, 3, 4, 5])),
  array([[[ nan,  nan,  nan,  nan,  nan,  nan],
          [ nan,  nan,  nan,  nan,  nan,  nan],
          [ nan,  nan,  nan,  nan,  nan,  nan],
          ..., 
          [ nan,  nan,  nan,  nan,  nan,  nan],
          [ nan,  nan,  nan,  nan,  nan,  nan],
          [ nan,  nan,  nan,  nan,  nan,  nan]],
  
         [[ nan,  nan,  nan,  nan,  nan,  nan],
          [ nan,  nan,  nan,  nan,  nan,  nan],
          [ nan,  nan,  nan,  nan, 

In [ ]:
data[1]['Euclidean distance']

In [37]:
meas = bench.measure()

Started at:  Jul 01 2017 20:47:51


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


First sweep...

D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:637: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


Time left: 0 h 0 m 0.0 s,	parameter values: Interleaving sequence id:      9,	average cycle time:   33s	s	
Elapsed time: 0 h 5 m 25.97 s


In [58]:
meas['Xo']

(('Interleaving sequence id',),
 (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),),
 array([ 0.0223 ,  0.14825, -0.23595, -0.0472 , -0.02075,  0.1296 ,
         0.12835,  0.21955,  0.1487 , -0.1806 ]),
 {})

In [73]:
import plotting
imp.reload(plotting)
plotting.plot_measurement(bench.reference_benchmark_result)

c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


{'-Xo': {'axes': <matplotlib.axes._axes.Axes at 0xd613b38>,
  'plots': {'-Xo': {'filter': <function plotting.plot_measurement.<locals>.<lambda>>,
    'mname': '-Xo',
    'plot': <matplotlib.lines.Line2D at 0x141cea58>}}},
 '-Yo': {'axes': <matplotlib.axes._axes.Axes at 0xd0e7e80>,
  'plots': {'-Yo': {'filter': <function plotting.plot_measurement.<locals>.<lambda>>,
    'mname': '-Yo',
    'plot': <matplotlib.lines.Line2D at 0xcfdf908>}}},
 'Euclidean distance': {'axes': <matplotlib.axes._axes.Axes at 0x15c17898>,
  'plots': {'Euclidean distance': {'filter': <function plotting.plot_measurement.<locals>.<lambda>>,
    'mname': 'Euclidean distance',
    'plot': <matplotlib.lines.Line2D at 0xcec84a8>}}},
 'Xo': {'axes': <matplotlib.axes._axes.Axes at 0xe518160>,
  'plots': {'Xo': {'filter': <function plotting.plot_measurement.<locals>.<lambda>>,
    'mname': 'Xo',
    'plot': <matplotlib.lines.Line2D at 0x129076d8>}}},
 'Yo': {'axes': <matplotlib.axes._axes.Axes at 0xe2f4898>,
  'plots': {

# Optimizing Drag

In [25]:
num_interleved_pulses = 2
alphas = np.linspace(-1e-8, 1e-8, 201)
def set_drag (alpha):
    ramsey_half = [pg.p('iq_ex', length_hd, pg.gauss_hd, amp_x_hd, 0, sigma_hd, alpha), 
                 pg.p(None, pause_length_hd)]
    interleaved_identity = [pg.p('iq_ex', length_hd, pg.gauss_hd, -amp_x_hd, 0, sigma_hd, alpha), 
                           pg.p(None, pause_length_hd),
                           pg.p('iq_ex', length_hd, pg.gauss_hd, amp_x_hd, 0, sigma_hd, alpha), 
                           pg.p(None, pause_length_hd)]
    tomo.set_prepare_seq(ramsey_half+interleaved_identity*num_interleved_pulses+ramsey_half)
    
measurement = sweep.sweep(tomo, (alphas, set_drag, 'delta_freq'), filename='DRAG', output=False)

Started at:  Jun 29 2017 21:21:39


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


First sweep...

D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:637: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


Time left: 0 h 0 m 0.0 s,	parameter values: delta_freq:  1e-08,	average cycle time:   10s	0s			
Elapsed time: 0 h 33 m 41.48 s


### Randomized Benchmarking

In [157]:
pause_pulse = tomography.pause(pause_length, awg_channels)
pulses = {0:tomography.ex_gauss_hd(amp_x_hd, amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          1:tomography.ex_gauss_hd(amp_y_hd, -amp_x_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          2:tomography.ex_gauss_hd(-amp_x_hd, -amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          3:tomography.ex_gauss_hd(-amp_y_hd, amp_x_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi)}

unitaries = {0:np.sqrt(0.5)*np.asarray([[1, -1j],  [-1j, 1]]),\
             1:np.sqrt(0.5)*np.asarray([[1, -1],    [1, 1]]),\
             2:np.sqrt(0.5)*np.asarray([[1, 1j],   [1j, 1]]),\
             3:np.sqrt(0.5)*np.asarray([[1, 1],   [-1, 1]])}

observables = { 'X': 0.5*np.asarray([[0, 1],   [1, 0]]),
                'Y': 0.5*np.asarray([[0, -1j],   [1j, 0]]),
                '-X': 0.5*np.asarray([[0, -1],   [-1, 0]]),
                '-Y': 0.5*np.asarray([[0, 1j],   [-1j, 0]]),
                'Z': 0.5*np.asarray([[1, 0],   [0, -1]])}

initial = np.asarray([0, 1]).T
random_pulse_ids = np.arange(20)
num_gates = np.linspace(0, 100, 11)

sequences = []
pulse_sequences = []
theor_projs = {o:np.zeros((len(num_gates), len(random_pulse_ids))) for o in observables.keys()}
rho_t = np.zeros((len(num_gates), len(random_pulse_ids),2,2), dtype=np.complex)
psis = np.zeros((len(num_gates), len(random_pulse_ids),2), dtype=np.complex)
n=0

def set_num_gates(n_new):
    global n
    n = n_new

def set_random_sequence(id):
    sequence = np.random.randint (4, size = n).tolist()
    sequence_pulses = [j for i in [[pulses[i], pause_pulse] for i in sequence] for j in i]
    sequence_unitaries = [unitaries[i] for i in sequence]
    psi = initial.copy()
    
    for U in sequence_unitaries:
        psi = np.dot(U, psi)
    
    rho_t[n,id,:,:] = np.einsum('i,j->ij', np.conj(psi), psi)
    psis[n,id,:] = psi
    
    sequences.append(sequence)
    for o in observables.keys():
        theor_projs[o][n,id] = np.dot(np.dot(np.conj(psi), observables[o]), psi)
    
    tomo_hd.set_prepare_seq(sequence_pulses)
    pulse_sequences.append(sequence_pulses)
    
m = sweep.sweep(tomo_hd, (num_gates, set_num_gates, 'gate num'), (random_pulse_ids, set_random_sequence, 'random seq id'), filename='RB', output=False)
X = (m['X'][2]-m['-X'][2])/2.
Y = (m['Y'][2]-m['-Y'][2])/2.
Z = m['Z'][2]
rho_m = np.transpose([[0.5+Z, X+1j*Y], [X-1j*Y, 0.5-Z]], [2,3,0,1])
fidelities = np.sqrt(np.einsum('nmi,nmji,nmj->nm', np.conj(psis), rho_m, psis))

m['Fidelity'] = [d for d in m['X']]
m['Fidelity'][2] = np.asarray(fidelities, dtype=np.float)
for o,O in observables.items():
    m[o+' fit'] = [d for d in m[o]]
    m[o+' fit'][2] = np.asarray(theor_projs[o], dtype=np.float)
#m['rho_t'] = m['X'].copy()
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, m)

Started at:  Apr 28 2017 16:50:15


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


First sweep...

c:\python36\lib\site-packages\ipykernel_launcher.py:44: ComplexWarning: Casting complex values to real discards the imaginary part


Time left: 0 h 0 m 0.0 s,	parameter values: gate num:     20,	random seq id:     19,	average cycle time:   14s				
Elapsed time: 1 h 41 m 29.59 s


c:\python36\lib\site-packages\numpy\core\numeric.py:531: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [ ]:
plt.plot(m['Fidelity'][1]m['Fidelity'])

In [154]:
rho_m

array([[[[ -4.54105167e-03+0.j        ,  -6.70718763e-03-0.01268007j],
         [ -6.70718763e-03+0.01268007j,   1.00454105e+00+0.j        ]],

        [[  1.90080873e-02+0.j        ,  -7.31600466e-04-0.00128394j],
         [ -7.31600466e-04+0.00128394j,   9.80991913e-01+0.j        ]]],


       [[[  5.23648939e-01+0.j        ,   1.67812796e-02+0.45229873j],
         [  1.67812796e-02-0.45229873j,   4.76351061e-01+0.j        ]],

        [[  5.34648328e-01+0.j        ,   9.83864179e-03+0.46630912j],
         [  9.83864179e-03-0.46630912j,   4.65351672e-01+0.j        ]]]])

In [168]:
plt.plot(m['Fidelity'][1][0], np.mean(m['Fidelity'][2],axis=1))
plt.xlabel('gate num')
plt.ylabel('fidelity')
plt.grid()

In [152]:
rho_m = np.transpose([[0.5+Z, X+1j*Y], [X-1j*Y, 0.5-Z]], [2,3,0,1])
fidelities = np.sqrt(np.einsum('mni,mnji,mnj->mn', np.conj(psis), rho_m, psis))

m['Fidelity'] = [d for d in m['X']]
m['Fidelity'][2] = np.asarray(fidelities, dtype=np.float)
for o,O in observables.items():
    m[o+' fit'] = [d for d in m[o]]
    m[o+' fit'][2] = np.asarray(theor_projs[o], dtype=np.float)
#m['rho_t'] = m['X'].copy()
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, m)

c:\python36\lib\site-packages\numpy\core\numeric.py:531: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


In [143]:
m

{'-X': (('random seq id', 'gate num'),
  (array([0, 1]), array([0, 1])),
  array([[ 0.02071368,  0.06280285],
         [-0.47004341, -0.47692684]]),
  {}),
 '-X fit': [('random seq id', 'gate num'),
  (array([0, 1]), array([0, 1])),
  array([[ 0. ,  0. ],
         [ 0. , -0.5]]),
  {}],
 '-Y': (('random seq id', 'gate num'),
  (array([0, 1]), array([0, 1])),
  array([[ 0.02339924,  0.07005866],
         [ 0.01224385,  0.00593551]]),
  {}),
 '-Y fit': [('random seq id', 'gate num'),
  (array([0, 1]), array([0, 1])),
  array([[ 0.,  0.],
         [ 0.,  0.]]),
  {}],
 'Fidelity': [('random seq id', 'gate num'),
  (array([0, 1]), array([0, 1])),
  array([[ 0.99258696,  0.        ],
         [ 0.        ,  0.99811094]]),
  {}],
 'X': (('random seq id', 'gate num'),
  (array([0, 1]), array([0, 1])),
  array([[ 0.03073913,  0.03102173],
         [ 0.50264322,  0.51552406]]),
  {}),
 'X fit': [('random seq id', 'gate num'),
  (array([0, 1]), array([0, 1])),
  array([[ 0. ,  0. ],
         [ 0

In [122]:
fidelities

array([ 0.97991445+0.j,  0.93312074+0.j,  0.97351467+0.j,  0.88195572+0.j,
        0.98395691+0.j])

In [123]:
psis

[array([-0.5+0.5j, -0.5-0.5j]),
 array([ 0.5-0.5j, -0.5-0.5j]),
 array([ 0.5+0.5j,  0.5+0.5j]),
 array([ 0.5-0.5j, -0.5+0.5j]),
 array([-0.5+0.5j, -0.5+0.5j])]

In [124]:
rho_m

array([[[ 0.59661779+0.j        ,  0.11373132+0.46023232j],
        [ 0.11373132-0.46023232j,  0.40338221+0.j        ]],

       [[ 0.54508677+0.j        ,  0.06986417-0.37071432j],
        [ 0.06986417+0.37071432j,  0.45491323+0.j        ]],

       [[ 0.68233096+0.j        ,  0.44773082+0.08804433j],
        [ 0.44773082-0.08804433j,  0.31766904+0.j        ]],

       [[ 0.45918886+0.j        , -0.27784590+0.18198637j],
        [-0.27784590-0.18198637j,  0.54081114+0.j        ]],

       [[ 0.56393048+0.j        ,  0.46817119+0.00919325j],
        [ 0.46817119-0.00919325j,  0.43606952+0.j        ]]])

In [125]:
rho_t

array([[[ 0.5+0.j ,  0.0+0.5j],
        [ 0.0-0.5j,  0.5+0.j ]],

       [[ 0.5+0.j ,  0.0-0.5j],
        [ 0.0+0.5j,  0.5+0.j ]],

       [[ 0.5+0.j ,  0.5+0.j ],
        [ 0.5+0.j ,  0.5+0.j ]],

       [[ 0.5+0.j , -0.5+0.j ],
        [-0.5+0.j ,  0.5+0.j ]],

       [[ 0.5+0.j ,  0.5+0.j ],
        [ 0.5+0.j ,  0.5+0.j ]]])

In [198]:
measurement['theory'] = thoer_proj
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, measurement)

KeyError: 'Sequences'

In [201]:
print(theor_projs)
print(measurement)
print(psis)
print (sequences)
print (rho_t)
print (rho_m)

[{'X': 0j, 'Y': 0j, '-X': 0j, '-Y': 0j, 'Z': (0.50000000000000022+0j)}, {'X': 0j, 'Y': 0j, '-X': 0j, '-Y': 0j, 'Z': (0.50000000000000022+0j)}, {'X': (0.50000000000000022+0j), 'Y': 0j, '-X': (-0.50000000000000022+0j), '-Y': 0j, 'Z': 0j}, {'X': 0j, 'Y': (-0.50000000000000022+0j), '-X': 0j, '-Y': (0.50000000000000022+0j), 'Z': 0j}, {'X': 0j, 'Y': (-0.50000000000000022+0j), '-X': 0j, '-Y': (0.50000000000000022+0j), 'Z': 0j}, {'X': (-0.50000000000000022+0j), 'Y': 0j, '-X': (0.50000000000000022+0j), '-Y': 0j, 'Z': 0j}, {'X': 0j, 'Y': (-0.50000000000000022+0j), '-X': 0j, '-Y': (0.50000000000000022+0j), 'Z': 0j}, {'X': 0j, 'Y': 0j, '-X': 0j, '-Y': 0j, 'Z': (-0.50000000000000022+0j)}, {'X': (0.50000000000000022+0j), 'Y': 0j, '-X': (-0.50000000000000022+0j), '-Y': 0j, 'Z': 0j}, {'X': (0.50000000000000022+0j), 'Y': 0j, '-X': (-0.50000000000000022+0j), '-Y': 0j, 'Z': 0j}]
{'X': (('random seq id',), (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),), array([ 0.13529069,  0.03693103,  0.46135948,  0.09578221,

In [194]:
np.dot(np.conj(psis[1]), np.dot(observables['Y'],psis[1]))

(-0.50000000000000022+0j)

In [71]:
num_pulses = np.round(np.arange(0, 41)).astype(np.int)

def set_num_pulses(n):
    tomoz.set_prepare_seq([tomography.ex_gauss(amp_x, length, sigma, awg_channels), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomoz, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

Started at:  Apr 27 2017 16:48:18
First sweep...

c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: SY pi2 pulse num:     40,	average cycle time:  3.5s			
Elapsed time: 0 h 2 m 24.22 s


In [115]:
num_pulses = np.round(np.arange(0, 21)).astype(np.int)

def set_num_pulses(n):
    tomo.set_prepare_seq([tomography.ex_gauss_hd(amp_x_hd, amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomo, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

Started at:  Apr 27 2017 20:15:31


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: SX pi2 pulse num:     20,	average cycle time:   18s	s	
Elapsed time: 0 h 6 m 16.75 s


In [110]:
num_pulses = np.round(np.arange(0, 21)).astype(np.int)

def set_num_pulses(n):
    tomo.set_prepare_seq([tomography.ex_gauss(amp_x_hd, length_hd, sigma_hd, awg_channels), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomo, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

Started at:  Apr 27 2017 19:39:12


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: SX pi2 pulse num:     20,	average cycle time:   18s	s	
Elapsed time: 0 h 6 m 14.09 s


In [79]:
ex_amplitude=0.25
lengths = np.linspace(0e-9, 200e-9, 51)
readout_begin = np.max(lengths)

def set_ex_length(length):
    tomo.set_prepare_seq([tomography.ex_rect(ex_amplitude, length, awg_channels), tomography.pause(pause_length, awg_channels)])

measurement = sweep.sweep(tomo, (lengths, set_ex_length, 'Rabi x-axis pulse length'), filename='Rabi', output=False)

Started at:  Apr 27 2017 17:39:03


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: Rabi x-axis pulse length:  3e-07,	average cycle time:   18s	18s	
Elapsed time: 0 h 15 m 21.47 s


In [85]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(measurement ['X'][2], measurement ['Y'][2], measurement ['Z'][2], c='r', marker='o')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_xlim([-0.6, 0.6])
ax.set_ylim([-0.6, 0.6])
ax.set_zlim([-0.6, 0.6])

(-0.6, 0.6)

In [57]:
ex_amplitude = 0.25
pause_length = 16e-9
lengths = np.linspace(0e-9, 2000e-9, 2001)
readout_begin = np.max(lengths)

def set_ex_length(length):
    tomo.set_prepare_seq([tomography.ex_rect(1j*ex_amplitude, length, awg_channels), tomography.pause(pause_length, awg_channels)])

measurement = sweep.sweep(tomo, (lengths, set_ex_length, 'Rabi y-axis pulse length'), filename='Rabi', output=False)

Started at:  Apr 27 2017 00:17:26


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: Rabi y-axis pulse length:  2e-06,	average cycle time:   10s	 10s		
Elapsed time: 5 h 33 m 44.24 s


### Optimizing phase between x and y pulses

In [111]:
c_phase = 0
c_n = 0

adc.set_software_averages(1)
adc.set_software_nums_multi(3)

def set_n(n):
    global c_n
    global c_phase
    c_n = n 
    y_pulse = [tomography.ex_gauss_hd(amp_x_hd*np.exp(1j*c_phase), amp_y_hd*np.exp(1j*c_phase), length_hd, sigma_hd, awg_channels, -200e6), tomography.pause(pause_length, awg_channels)]
    tomo.set_prepare_seq(y_pulse*c_n)

def set_y_phase(phase):
    global c_n
    global c_phase
    c_phase = phase
    y_pulse = [tomography.ex_gauss_hd(amp_x_hd*np.exp(1j*c_phase), amp_y_hd*np.exp(1j*c_phase), length_hd, sigma_hd, awg_channels, -200e6), tomography.pause(pause_length, awg_channels)]
    tomo.set_prepare_seq(y_pulse*c_n)

phases = np.linspace(np.pi/2.*0.8, np.pi/2.*1.2, 11, endpoint=False)
num_pulses = np.arange(0,5)
measurement = sweep.sweep(tomo, (phases, set_y_phase, 'y-pulse phase'), (num_pulses, set_n, 'y pulse num'), filename='y-phase', output=False)

Started at:  Apr 27 2017 19:45:43


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 3 m 54.74 s,	parameter values: y-pulse phase:  1.714,	y pulse num:      1,	average cycle time:   18s		

KeyboardInterrupt: 

In [73]:
phases[7]

1.6564761264382546

In [117]:
plt.plot(np.asarray(results).T)

c:\python36\lib\site-packages\numpy\core\numeric.py:531: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [139]:
adc.stop()

In [88]:
plt.close()

# load calibration data

In [19]:
import pickle
f0 = open('D:\\qtlab\\ReiData\\data\\2017-04-26\\13-11-46\\Voltage ro calibration.pkl', 'rb')
f1 = open('D:\\qtlab\\ReiData\\data\\2017-04-26\\13-11-14\\Voltage ro calibration.pkl', 'rb')
cal0 = pickle.load(f0)
cal1 = pickle.load(f1)

In [20]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt4
cal0_nomean = cal0[1]['Voltage'][2][0,:,:]-np.mean(cal0[1]['Voltage'][2][0,:,:])
cal1_nomean = cal0[1]['Voltage'][2][1,:,:]-np.mean(cal0[1]['Voltage'][2][1,:,:])

cal0_mean = np.mean(cal0_nomean, axis=0)
cal1_mean = np.mean(cal1_nomean, axis=0)

In [21]:
cal_mean = (cal0_mean+cal1_mean)/2.
cal_diff = (cal1_mean-cal0_mean)

plt.figure('Samples')
plt.plot(np.mean(cal0_nomean, axis=0), label='|0>')
plt.plot(np.mean(cal1_nomean, axis=0), label='|1>')
plt.legend()

plt.figure('Offset & feature')
plt.plot(cal_mean, label='mean')
plt.plot(cal_diff, label='diff')
plt.legend()

c:\python36\lib\site-packages\numpy\core\numeric.py:531: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [22]:
plt.figure('Fourier domain')
plt.plot(np.abs(np.fft.fft(cal0_mean)), label='|0>')
plt.plot(np.abs(np.fft.fft(cal1_mean)), label='|1>') 
plt.plot(np.abs(np.fft.fft(cal_mean)), label='mean')
plt.plot(np.abs(np.fft.fft(cal_diff)), label='diff')
plt.legend()

feature = np.conj(cal_diff/np.sum(np.abs(cal_diff**2)))

In [23]:
coeffs0 = np.dot(cal0_nomean-cal_mean, feature)
coeffs1 = np.dot(cal1_nomean-cal_mean, feature)
predictions = np.asarray([np.real(coeffs0), np.real(coeffs1)])

In [24]:
hist_all, bins = np.histogram(predictions, bins='auto')
proba_points = (bins[1:]+bins[:-1])/2.
hists = []
for y in range(2):
    hists.append(np.histogram(predictions[y,:], bins=bins)[0])

hists = np.asarray(hists, dtype=float)
probabilities = hists/hist_all
naive_probabilities = np.asarray([proba_points<0, proba_points>0], dtype=float)
probabilities[np.isnan(probabilities)] = naive_probabilities[np.isnan(probabilities)]
predictor = lambda x: np.interp(x, proba_points, probabilities[1,:], left=0., right=1.)
calib_proba_points = proba_points
calib_proba = probabilities[1,:]
calib_hists = hists

In [25]:
from sklearn.metrics import roc_curve, roc_auc_score
roc = roc_curve([0]*coeffs0.size+[1]*coeffs0.size, predictor(predictions.ravel()))
roc_auc = roc_auc_score([0]*coeffs0.size+[1]*coeffs0.size, predictor(predictions.ravel()))

In [44]:
plt.figure('ROC')
plt.plot(roc[0], roc[1])
plt.xlabel('True Positive Rate')
plt.ylabel('True Negative Rate')
roc_auc

0.74963895749999998

In [27]:
plt.figure('probability curve')
plt.plot(calib_proba_points, calib_proba)

In [41]:
plt.figure('Readout hists')
plt.bar(proba_points-0.25, hists[0], width=0.05, label='|0>')
plt.bar(proba_points+0.25, hists[1], width=0.05, label='|1>')
plt.legend()

In [ ]:
plt.close()

In [62]:
cal0_nomean.shape

(20000, 1024)

In [76]:
plt.figure(figsize=(12,3))
plt.imshow(np.sum(np.real(np.reshape(cal0_nomean, (200, 100, 1024))), axis=0), cmap='RdBu')
plt.figure(figsize=(12,3))
plt.imshow(np.sum(np.real(np.reshape(cal1_nomean, (200, 100, 1024))), axis=0), cmap='RdBu')

In [57]:
plt.plot(np.mean(cal0_nomean, axis=0))
plt.plot(np.mean(cal1_nomean, axis=0))

c:\python36\lib\site-packages\numpy\core\numeric.py:531: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
